In [1]:
import yaml
import collections

import pandas as pd
import numpy as np

In [2]:
def read_init_file(init_file_name):
    """Reads in the model specification from yaml file"""

    # Import yaml initialization file as dictionary init_dict
    if isinstance(init_file_name, str):
        with open(init_file_name) as y:
            init_dict = yaml.load(y, Loader=yaml.FullLoader)
    else:
        init_dict = init_file_name

    init_dict = expand_init_dict(init_dict)

    model_spec = create_namedtuple(init_dict)

    return model_spec


def flatten_init_dict(init_dict):
    """Removes the grouping from the nested dictionary"""

    groups = [
        "GENERAL",
        "CONSTANTS",
        "INITIAL_CONDITIONS",
        "SIMULATION",
        "SOLUTION",
    ]

    model_spec = dict()

    for group in groups:

        keys_ = list(init_dict[group].keys())
        values_ = list(init_dict[group].values())

        for k_, key_ in enumerate(keys_):

            model_spec[key_] = values_[k_]

    return model_spec


def dict_to_namedtuple(dictionary):
    """Coverts non-nested dictionary to namedtuple"""

    return collections.namedtuple("model_specification", dictionary.keys())(**dictionary)



Import model spec
-----------------

In [3]:
# Import yaml initialization file for model spec/options as dictionary init_dict_spec

with open('options_init.yml') as y:
    init_dict_spec = yaml.load(y, Loader=yaml.FullLoader)

In [4]:
init_dict_spec

{'GENERAL': {'num_periods': 30},
 'CONSTANTS': {'delta': 0.98, 'mu': -0.56, 'benefits': 6.0},
 'INITIAL_CONDITIONS': {'educ_max': 16, 'educ_min': 10},
 'SIMULATION': {'seed_sim': 102, 'num_agents_sim': 1000},
 'SOLUTION': {'seed_emax': 635, 'num_draws_emax': 500}}

Extract model specification parameters as namedtuple
-----------------------------------------------------------------

In [5]:
model_spec = flatten_init_dict(init_dict_spec)

In [6]:
model_spec = dict_to_namedtuple(model_spec)

In [7]:
model_spec

model_specification(num_periods=30, delta=0.98, mu=-0.56, benefits=6.0, educ_max=16, educ_min=10, seed_sim=102, num_agents_sim=1000, seed_emax=635, num_draws_emax=500)

Extract estimation parameters as multiindex data frame
------------------------------------------------------------------

In [8]:
model_params_df = pd.read_csv("params_init.csv")

In [9]:
model_params_df

,category,name,value,fixed,lower,upper
0,const_wage_eq,gamma_0s1,1.792,'False',1.000,3.000
1,const_wage_eq,gamma_0s2,1.808,'False',1.000,3.000
2,const_wage_eq,gamma_0s3,1.856,'False',1.000,3.000
3,exp_returns,gamma_1s1,0.122,'False',0.050,0.400
4,exp_returns,gamma_1s2,0.199,'False',0.050,0.400
5,exp_returns,gamma_1s3,0.266,'False',0.050,0.400
6,exp_accm,g_s1,0.200,'False',0.005,0.600
7,exp_accm,g_s2,0.200,'False',0.005,0.600
8,exp_accm,g_s3,0.200,'False',0.005,0.600
9,exp_deprec,delta_s1,0.081,'False',0.001,0.150


In [10]:
model_params_df.set_index(["category", "name"], inplace=True)

In [11]:
model_params_df

value    fixed  lower  upper
category      name                                   
const_wage_eq gamma_0s1  1.792  'False'  1.000  3.000
              gamma_0s2  1.808  'False'  1.000  3.000
              gamma_0s3  1.856  'False'  1.000  3.000
exp_returns   gamma_1s1  0.122  'False'  0.050  0.400
              gamma_1s2  0.199  'False'  0.050  0.400
              gamma_1s3  0.266  'False'  0.050  0.400
exp_accm      g_s1       0.200  'False'  0.005  0.600
              g_s2       0.200  'False'  0.005  0.600
              g_s3       0.200  'False'  0.005  0.600
exp_deprec    delta_s1   0.081  'False'  0.001  0.150
              delta_s2   0.057  'False'  0.001  0.150
              delta_s3   0.073  'False'  0.001  0.150
diutil_work   const_p    1.880  'False'  1.000  4.000
              const_f    2.330  'False'  1.000  4.000
hetrg_unobs   theta_p1  -0.200  'False' -0.400 -0.050
              theta_f1  -0.500  'False' -0.800 -0.150
shares        share_1    0.500  'False'  0.001  0.999
sd_wage_shock sigma_1    0.010  'False'  0.001  0.800
              sigma_2    0.200  'False'  0.001  0.800
              sigma_3    0.400  'False'  0.001  0.800

In [12]:
model_params = model_params_df["value"]

In [13]:
model_params

category       name     
const_wage_eq  gamma_0s1    1.792
               gamma_0s2    1.808
               gamma_0s3    1.856
exp_returns    gamma_1s1    0.122
               gamma_1s2    0.199
               gamma_1s3    0.266
exp_accm       g_s1         0.200
               g_s2         0.200
               g_s3         0.200
exp_deprec     delta_s1     0.081
               delta_s2     0.057
               delta_s3     0.073
diutil_work    const_p      1.880
               const_f      2.330
hetrg_unobs    theta_p1    -0.200
               theta_f1    -0.500
shares         share_1      0.500
sd_wage_shock  sigma_1      0.010
               sigma_2      0.200
               sigma_3      0.400
Name: value, dtype: float64

Example usage in functions
--------------------------------

In [14]:
# Set example values
states = [1, 12, 2, 0, 1, 1]
covariates = 12

In [16]:
def calculate_log_wage_systematic(model_spec, model_params, states, covariates):
    """Calculate systematic wages, i.e., wages net of shock, for all states."""

    exp_p, exp_f = states[3], states[4]
    educ_level = covariates - model_spec.educ_min

    # Construct wage components
    gamma_0s = model_params.loc[('const_wage_eq')].to_numpy()[educ_level]
    gamma_1s = model_params.loc[('const_wage_eq')].to_numpy()[educ_level]
    period_exp_sum = exp_p * model_params.loc[('exp_accm')].to_numpy()[educ_level] + exp_f
    depreciation = 1 - model_params.loc[('exp_deprec')].to_numpy()[educ_level]

    # Calculate wage in the given state
    period_exp_total = period_exp_sum * depreciation + 1
    returns_to_exp = gamma_1s * np.log(period_exp_total)
    log_wage_systematic = gamma_0s + returns_to_exp

    return log_wage_systematic

In [17]:
test = calculate_log_wage_systematic(model_spec, model_params, states, covariates)
test

3.0734699067862175